In [1]:
import os
import sys
import random
import pstats

from profile import Profile as PythonProfiler

import pandas as pd
import hippiehug

sys.path.append('..')

from claimchain import State, View, LocalParams
from claimchain.utils.profiling import Profiler

## Generating test data

This simulates `nb_readers` (200 by default) readers, each with own set of cryptographic keys. Every reader gets a random **32 byte** label associated with them, and **512 bytes** randomly generated claim about them.

The random capability matrix is generated: for each of the target readers _t_, `nb_caps_per_reader` (5 by default) other readers are randomly chosen. These are the ones that the reader _t_ will be able to access claims about, i.e. be included in _t_'s capability list.



In [2]:
def generate_test_data(nb_readers=200, nb_caps_per_reader=1, label_size=32, claim_size=512):
    labels = [b'label_' + os.urandom(label_size - 6) for _ in range(nb_readers)]
    claims = [b'claim_' + os.urandom(claim_size - 6) for _ in range(nb_readers)]

    params = [LocalParams.generate() for _ in range(nb_readers)]

    reader_graph = {}
    for reader_index in range(nb_readers):
        cap = random.sample(range(nb_readers), nb_caps_per_reader)
        reader_graph[reader_index] = cap

    return reader_graph, labels, claims, params

## Simulating the owner
The owner builds a state consisting of the claims generated above, with the access capabilities as generated above. The state is committed to the chain.

In [3]:
def commit_claims(reader_graph, labels, claims, params):
    owner_params = LocalParams.generate()
    with owner_params.as_default():
        state = State()
        
        # Add claims
        for label, claim in zip(labels, claims):
            state[label] = claim
        
        # Define the capability lists
        for reader_index, cap_indexes in reader_graph.items():
            reader_dh_pk = params[reader_index].dh.pk
            cap_labels = [labels[cap_index] for cap_index in cap_indexes]
            state.grant_access(reader_dh_pk, cap_labels)
        
        # Commit
        chain = hippiehug.Chain()
        state.commit(target_chain=chain)

        return chain, state

## Simulating the readers

In [4]:
def read_claims(chain, reader_graph, labels, claims, params):
    # Go over all of the readers
    for reader_index, caps_indexes in reader_graph.items():
        reader_params = params[reader_index]
        with reader_params.as_default():
            view = View(chain)
            # Retrieve each of the accessible labels
            for cap_index in caps_indexes:
                label = labels[cap_index]
                read_claim = view[labels[cap_index]]
                
                # Check the value matches original to be sure
                assert read_claim == claims[cap_index]

## Measuring timings

In [5]:
def simulate():
    reader_graph, labels, claims, params = generate_test_data()

    # Commit claims
    chain, state = commit_claims(reader_graph, labels, claims, params)
    # Simulate readers
    read_claims(chain, reader_graph, labels, claims, params)

### High-level profiling

In [6]:
profiler = Profiler()
with profiler.as_default():
    simulate()
    stats = profiler.compute_stats()

In [13]:
df = pd.DataFrame(stats)
df = df.T
stat_cols = ['avg', 'max', 'min', 'std']
df = df[stat_cols].apply(lambda value: value * 1000)  # milliseconds
df

,avg,max,min,std
_build_tree,15.187979,15.187979,15.187979,NaN
_lookup_capability,0.276275,0.309706,0.269651,0.005330
_lookup_claim,2.465253,3.217697,1.965046,0.194645
decode_capability,0.136579,0.156164,0.134468,0.002812
decode_claim,2.451662,3.199577,1.952171,0.194462
encode_capability,0.134870,0.147343,0.133514,0.001480
encode_claim,1.536522,2.870321,1.180887,0.213679


In [14]:
print(df.sort_values(by='avg').to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &        avg &        max &        min &       std \\
\midrule
encode\_capability  &   0.134870 &   0.147343 &   0.133514 &  0.001480 \\
decode\_capability  &   0.136579 &   0.156164 &   0.134468 &  0.002812 \\
\_lookup\_capability &   0.276275 &   0.309706 &   0.269651 &  0.005330 \\
encode\_claim       &   1.536522 &   2.870321 &   1.180887 &  0.213679 \\
decode\_claim       &   2.451662 &   3.199577 &   1.952171 &  0.194462 \\
\_lookup\_claim      &   2.465253 &   3.217697 &   1.965046 &  0.194645 \\
\_build\_tree        &  15.187979 &  15.187979 &  15.187979 &       NaN \\
\bottomrule
\end{tabular}



### Low-level profiling

In [9]:
profiler = PythonProfiler()
profiler.runctx("simulate()", globals(), locals())
profiler.create_stats()

profiler.print_stats(sort='cumtime')

         544587 function calls (523310 primitive calls) in 2.722 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    2.722    2.722 profile:0(simulate())
        1    0.000    0.000    2.722    2.722 :0(exec)
        1    0.002    0.002    2.722    2.722 <string>:1(<module>)
        1    0.000    0.000    2.716    2.716 <ipython-input-5-8ff7cb198a40>:1(simulate)
        1    0.004    0.004    1.934    1.934 <ipython-input-4-1e1a01598d33>:1(read_claims)
 1201/801    0.007    0.000    1.388    0.002 profiling.py:34(wrapped)
      200    0.003    0.000    1.157    0.006 state.py:145(__init__)
     1800    0.006    0.000    1.033    0.001 encodings.py:45(ascii2pet)
      200    0.002    0.000    0.970    0.005 params.py:62(from_dict)
      800    0.006    0.000    0.967    0.001 params.py:70(maybe_load_keypair)
     1600    0.003    0.000    0.959    0.001 params.py:64(maybe_decode)
     1000    0.0